In [1]:
import traitlets
import os
from jetbot import Camera, bgr8_to_jpeg
import ipywidgets.widgets as widgets
from IPython.display import display
from uuid import uuid1
import json
import glob
import datetime

In [2]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

In [3]:
camera = Camera.instance(width=224, height=224)

image_widget = widgets.Image(format='jpeg', width=300, height=300)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# Label the 7 block configurations

In [6]:
THUMBNAIL_DIR = 'thumbnails'
BLOCKS_DATASET_DIR = 'dataset_blocks'

class_names = [
    'cross',
    'left',
    'right',
    'straight',
    't_left',
    't_right',
    't_straight'
]


def get_count(name):
    return len(glob.glob(os.path.join(BLOCKS_DATASET_DIR, name, '*.jpg')))


class Saver(object):
    
    def __init__(self, name, count_widget):
        self.name = name
        self.count_widget = count_widget
        
    def save(self, x):
        image_path = os.path.join(BLOCKS_DATASET_DIR, self.name, str(uuid1()) + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        self.count_widget.value = get_count(self.name)
        
    
# create thumbnail widgets
class_widgets = []
button_widgets = []
count_widgets = []
for i, name in enumerate(class_names):
    # create directory for class
    try:
        os.makedirs(os.path.join(BLOCKS_DATASET_DIR, name))
    except:
        pass
    
    # get thumbnail widget
    thumbnail_widget = widgets.Image(format='jpeg', width=80, height=80)
    thumbnail_path = os.path.join(THUMBNAIL_DIR, name + '.jpg')
    with open(thumbnail_path, 'rb') as f:
        thumbnail_widget.value = f.read()
    
    # get count widget
    count_layout = widgets.Layout(width='80px')
    count_widget = widgets.IntText(layout=count_layout)
    
    # get button
    button_layout = widgets.Layout(width='80px', height='50px')
    button_widget = widgets.Button(description=name, layout=button_layout)
    
    saver = Saver(name, count_widget)
    
    button_widget.on_click(saver.save)
    
    class_widget = widgets.VBox([
        thumbnail_widget,
        count_widget,
        button_widget,
    ])
    class_widgets.append(class_widget)

display(widgets.HBox(class_widgets))

In [7]:
!zip -r -q lego_city_{BLOCKS_DATASET_DIR}_{timestr()}.zip {BLOCKS_DATASET_DIR}